#### Training

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
gpt2.download_gpt2(model_name="124M")

In [ ]:
gpt2.mount_gdrive()

In [ ]:
file_name = "JSONtoTXT.txt"

In [ ]:
# import os
# os.chdir("/content/drive/MyDrive/IITK")
# !ls

In [ ]:
gpt2.copy_file_from_gdrive(file_name)
try:
  gpt2.copy_checkpoint_from_gdrive(run_name='LegalAnnotTextGen')
  print("Previous Checkpoint loaded")
except:
  print("New model to be trained")

In [ ]:
# tf.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=2000,
              restore_from='latest',
              run_name='LegalAnnotTextGen',
              print_every=200,
              sample_every=1000,
              save_every=500,
              overwrite = True,
              learning_rate=0.00001
              )


In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='LegalAnnotTextGen')

#### Generation of Text

In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime

2023-04-19 13:20:11.187269: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# gpt2.copy_checkpoint_from_gdrive(run_name='LegalAnnotTextGen')

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='LegalAnnotTextGen')

2023-04-19 13:20:12.821302: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 13:20:12.822286: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-19 13:20:12.822966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-19 13:20:14.437710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 13:20:14.438559: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 na

Loading checkpoint checkpoint/LegalAnnotTextGen/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/LegalAnnotTextGen/model-2000


2023-04-19 13:20:16.692435: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2023-04-19 13:20:16.726613: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


In [9]:
# sampleSize = 10
textPref = "<DOC><PREAMBLE>"
newContext = "<DOC><PREAMBLE>"
step = 0
while "</DOC>" not in newContext:
    prevContext = newContext
    newContext = gpt2.generate(sess,
                        run_name = 'LegalAnnotTextGen',
                        truncate='<END/>',
                        temperature=0.8,
                        prefix = prevContext,
                        include_prefix=False,
                        top_k = 50,
                        return_as_list=True)[0]
    
    if prevContext in newContext:
        step+=1
        if step>9:
            print(step)
            step = step%10
        newContext = newContext.replace(prevContext,'')
    textPref+=newContext
    newContext = newContext[int(0.5*len(newContext)):]

In [10]:
import re
textPref = re.sub('&','and',textPref)
print(textPref)

<DOC><PREAMBLE>THE COURT: VENKATARAMIAH, J. DELHI CENTRAL </PREAMBLE><PREAMBLE>DISTRICT :</PREAMBLE><PREAMBLE>WRIT PETITION NO.3055 OF 2010</PREAMBLE><PREAMBLE> NARULAIAH, J.</PREAMBLE><PREAMBLE>WRIT PETITION NO.3056 OF 2010</PREAMBLE><PREAMBLE> VENKATARAMIAH, J. (DERIVED AS IN APPELLANT's case, CIT.</PREAMBLE><PREAMBLE>NOT JUDGMENT BEFORE R-C</PREAMBLE><PREAMBLE> 1. In R-C, the question is as to whether the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.</PREAMBLE><PREAMBLE>[617 D] 2. As held in the case of M/s. Fancy Chemicals Ltd. [1991] 2 SCC 699, the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.</PREAMBLE><PREAMBLE>3. As held in the case of Bentleys Ltd. [1988] 2 SCC 699, the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.</PREAMBLE><PREAMBLE>4. As held in the case of M/s. Bentleys Ltd. [1990] 2 SCC 1, the provisions of the Assam Amendment 

In [6]:
!pip install xmltodict

In [11]:
import xmltodict
import json

# Input XML string
xml_str = textPref

# Convert XML string to a dictionary
doc_dict = xmltodict.parse(xml_str)
# print(doc_dict)
# Extract the annotations from the dictionary
annotations = {}
for key, value in doc_dict['DOC'].items():
    if key.startswith('@'):
        # Ignore attributes like '@xmlns'
        continue
    annotations[key] = value

# Create the output JSON object
output = [{'id': 1, 'annotations': annotations}]

# Serialize the JSON object to a string
json_str = json.dumps(output)

# Print the JSON string
print(json_str)

# Save json File
with open("output.json", "w", encoding = "utf-8") as f:
  f.write(json_str)

[{"id": 1, "annotations": {"PREAMBLE": ["THE COURT: VENKATARAMIAH, J. DELHI CENTRAL", "DISTRICT :", "WRIT PETITION NO.3055 OF 2010", "NARULAIAH, J.", "WRIT PETITION NO.3056 OF 2010", "VENKATARAMIAH, J. (DERIVED AS IN APPELLANT's case, CIT.", "NOT JUDGMENT BEFORE R-C", "1. In R-C, the question is as to whether the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.", "[617 D] 2. As held in the case of M/s. Fancy Chemicals Ltd. [1991] 2 SCC 699, the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.", "3. As held in the case of Bentleys Ltd. [1988] 2 SCC 699, the provisions of the Assam Amendment Act, 1967, would attract cognizance from the District Court.", "4. As held in the case of M/s. Bentleys Ltd. [1990] 2 SCC 1, the provisions of the Assam Amendment Act, 1967, would bring cognizance against the directors of the directors' committee [or D.C.] of the Board of Directors [or the C.B.I.] for failure 